# Building a Gaming Community Support Bot with Discord and Elastic Agent Builder A2A

This notebook creates a Discord bot that connects to Elastic Agent Builder's A2A server. Players ask questions like "Who's the best Mage?" or "What's the current meta?" and get real-time answers powered by ES|QL analytics and semantic search.

## What you'll build

- **ES|QL tools** for structured analytics: leaderboards, hero stats, meta reports
- **Index Search tools** for unstructured knowledge: game mechanics, FAQs (semantic search)
- **An agent** that automatically selects the right tool based on the user's question

## Prerequisites

- Elasticsearch cluster (9.2+, or Serverless)
- Python 3.9+
- Discord bot token and server (for the final integration)

## Install dependencies

In [ ]:
!pip install elasticsearch requests

## Configuration

Set your environment variables:

```bash
export ELASTICSEARCH_URL="https://your-cluster.es.region.aws.elastic.co:443"
export ELASTIC_API_KEY="your-elasticsearch-api-key"
export KIBANA_URL="https://your-deployment.kb.region.aws.elastic.co"
export KIBANA_API_KEY="your-kibana-api-key"
```

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from datetime import datetime, timedelta
import requests
import random
import os

# Elasticsearch connection
es = Elasticsearch(
    hosts=[os.getenv("ELASTICSEARCH_URL")],
    api_key=os.environ["ELASTIC_API_KEY"],
    request_timeout=120,
)

# Verify connection
print(es.info())

## Create indices

Three indices serve different purposes:

| Index | Purpose | Query Type |
|-------|---------|------------|
| `player_stats` | Player profiles with wins, kills, rank | ES|QL analytics |
| `hero_meta` | Hero pick rates and win rates by tier | ES|QL analytics |
| `game_knowledge` | FAQs and game mechanics | Semantic search |

The `game_knowledge` index uses `semantic_text` for meaning-based search. Both `title` and `content` are copied into `semantic_field` for hybrid search.

In [ ]:
# Delete indices if they exist (for clean setup)
for index in ["player_stats", "hero_meta", "game_knowledge"]:
    if es.indices.exists(index=index):
        es.indices.delete(index=index)
        print(f"Deleted existing index: {index}")

In [ ]:
# Player stats index
es.indices.create(
    index="player_stats",
    mappings={
        "properties": {
            "player_id": {"type": "keyword"},
            "username": {"type": "keyword"},
            "hero": {"type": "keyword"},
            "wins": {"type": "integer"},
            "losses": {"type": "integer"},
            "kills": {"type": "integer"},
            "deaths": {"type": "integer"},
            "rank": {"type": "keyword"},
            "last_played": {"type": "date"},
        }
    },
)
print("Created player_stats index")

In [ ]:
# Hero meta index
es.indices.create(
    index="hero_meta",
    mappings={
        "properties": {
            "hero_name": {"type": "keyword"},
            "pick_rate": {"type": "float"},
            "win_rate": {"type": "float"},
            "tier": {"type": "keyword"},
            "patch_version": {"type": "keyword"},
        }
    },
)
print("Created hero_meta index")

In [ ]:
# Game knowledge index (for semantic search)
# Both title and content are copied into semantic_field for unified semantic search
es.indices.create(
    index="game_knowledge",
    mappings={
        "properties": {
            "title": {"type": "text", "copy_to": "semantic_field"},
            "content": {"type": "text", "copy_to": "semantic_field"},
            "category": {"type": "keyword"},
            "semantic_field": {
                "type": "semantic_text"
            },  # Semantic search queries this combined field
        }
    },
)
print("Created game_knowledge index")

## Index sample data

Sample gaming data includes:

- **5 players** with different heroes, ranks, and stats (kills, wins, losses)
- **5 heroes** with meta information (pick rate, win rate, tier ranking)
- **3 knowledge articles** covering common player questions (mounts, builds, ranking system)

In [ ]:
# Sample player data
players = [
    {
        "player_id": "p001",
        "username": "DragonSlayer99",
        "hero": "Warrior",
        "wins": 342,
        "losses": 198,
        "kills": 4521,
        "deaths": 2103,
        "rank": "Diamond",
    },
    {
        "player_id": "p002",
        "username": "ShadowMage",
        "hero": "Mage",
        "wins": 567,
        "losses": 234,
        "kills": 8932,
        "deaths": 3421,
        "rank": "Master",
    },
    {
        "player_id": "p003",
        "username": "HealBot3000",
        "hero": "Healer",
        "wins": 423,
        "losses": 187,
        "kills": 1234,
        "deaths": 1876,
        "rank": "Diamond",
    },
    {
        "player_id": "p004",
        "username": "TankMaster",
        "hero": "Tank",
        "wins": 298,
        "losses": 302,
        "kills": 2341,
        "deaths": 1543,
        "rank": "Platinum",
    },
    {
        "player_id": "p005",
        "username": "AssassinX",
        "hero": "Assassin",
        "wins": 789,
        "losses": 156,
        "kills": 12453,
        "deaths": 2987,
        "rank": "Grandmaster",
    },
]

for player in players:
    player["last_played"] = datetime.now() - timedelta(hours=random.randint(1, 72))

# Hero meta data
heroes = [
    {
        "hero_name": "Warrior",
        "pick_rate": 15.2,
        "win_rate": 51.3,
        "tier": "A",
        "patch_version": "2.4.1",
    },
    {
        "hero_name": "Mage",
        "pick_rate": 22.8,
        "win_rate": 54.7,
        "tier": "S",
        "patch_version": "2.4.1",
    },
    {
        "hero_name": "Healer",
        "pick_rate": 18.5,
        "win_rate": 52.1,
        "tier": "A",
        "patch_version": "2.4.1",
    },
    {
        "hero_name": "Tank",
        "pick_rate": 12.3,
        "win_rate": 48.9,
        "tier": "B",
        "patch_version": "2.4.1",
    },
    {
        "hero_name": "Assassin",
        "pick_rate": 31.2,
        "win_rate": 49.2,
        "tier": "A",
        "patch_version": "2.4.1",
    },
]

# Game knowledge for semantic search
knowledge = [
    {
        "title": "How to unlock the Dragon Mount",
        "content": "Complete the Dragon's Lair dungeon on Nightmare difficulty with all party members alive. The mount has a 15% drop rate.",
        "category": "mounts",
    },
    {
        "title": "Best Mage build for Season 4",
        "content": "Focus on Intelligence and Critical Chance. Use the Arcane Staff with Frost Runes. Prioritize cooldown reduction for burst damage.",
        "category": "builds",
    },
    {
        "title": "Understanding the ranking system",
        "content": "Ranks go from Bronze to Grandmaster. You need 100 points to advance. Wins give 25 points, losses subtract 20.",
        "category": "ranked",
    },
]

# Bulk index all data
actions = []
for player in players:
    actions.append({"_index": "player_stats", "_source": player})
for hero in heroes:
    actions.append({"_index": "hero_meta", "_source": hero})
for doc in knowledge:
    actions.append({"_index": "game_knowledge", "_source": doc})

success, errors = bulk(es, actions)
print(f"Indexed {success} documents")

es.indices.refresh(index="player_stats,hero_meta,game_knowledge")

## Kibana API connection

The Agent Builder tools and agent are created using the [Kibana API](https://www.elastic.co/docs/api/doc/kibana/). Find your Kibana URL in your Elastic Cloud deployment under **Applications > Kibana > Copy endpoint**.

In [ ]:
KIBANA_URL = os.environ["KIBANA_URL"]
KIBANA_API_KEY = os.environ["KIBANA_API_KEY"]

headers = {
    "kbn-xsrf": "true",
    "Authorization": f"ApiKey {KIBANA_API_KEY}",
    "Content-Type": "application/json",
}

## Create ES|QL tools

ES|QL tools handle **structured analytics queries**: aggregations, filtering, sorting on well-defined data.

Three tools cover different query types:

1. **leaderboard** - Top players by kills (no parameters)
2. **hero_stats** - Stats for a specific hero (dynamic `?hero` parameter)
3. **meta_report** - All heroes sorted by tier (no parameters)

The agent automatically selects the right tool based on the user's question.

In [ ]:
# Tool 1: Leaderboard
leaderboard_tool = {
    "id": "leaderboard",
    "type": "esql",
    "description": "Shows top players ranked by kills. Use when someone asks Who is the best? or Show me top players.",
    "configuration": {
        "query": """FROM player_stats
| STATS total_kills = SUM(kills), total_wins = SUM(wins) BY username, hero, rank
| SORT total_kills DESC
| LIMIT 10""",
        "params": {},
    },
}

response = requests.post(
    f"{KIBANA_URL}/api/agent_builder/tools", headers=headers, json=leaderboard_tool
)
print(f"Leaderboard tool: {response.status_code}")

In [ ]:
# Tool 2: Hero Stats (with dynamic parameter)
hero_stats_tool = {
    "id": "hero_stats",
    "type": "esql",
    "description": "Gets win rate, pick rate, and tier for a specific hero. Use when someone asks How good is Mage? or What is the win rate for Warrior?",
    "configuration": {
        "query": """FROM hero_meta
| WHERE hero_name == ?hero
| KEEP hero_name, win_rate, pick_rate, tier, patch_version""",
        "params": {
            "hero": {"type": "keyword", "description": "The hero name to look up"}
        },
    },
}

response = requests.post(
    f"{KIBANA_URL}/api/agent_builder/tools", headers=headers, json=hero_stats_tool
)
print(f"Hero stats tool: {response.status_code}")

In [ ]:
# Tool 3: Meta Report
meta_report_tool = {
    "id": "meta_report",
    "type": "esql",
    "description": "Shows all heroes sorted by tier and win rate. Use when someone asks What is the current meta? or Which heroes are S-tier?",
    "configuration": {
        "query": """FROM hero_meta
| SORT tier ASC, win_rate DESC
| KEEP hero_name, tier, win_rate, pick_rate""",
        "params": {},
    },
}

response = requests.post(
    f"{KIBANA_URL}/api/agent_builder/tools", headers=headers, json=meta_report_tool
)
print(f"Meta report tool: {response.status_code}")

## Create Index Search tool

Index Search tools handle **unstructured knowledge queries** using semantic search. Unlike ES|QL which queries structured fields, Index Search understands meaning and context.

A user asking "How do I get the dragon mount?" matches "How to unlock the Dragon Mount" even without exact keyword matches—semantic search understands they mean the same thing.

In [ ]:
game_knowledge_tool = {
    "id": "game_knowledge",
    "type": "index_search",
    "description": "Searches game guides, FAQs, and mechanics. Use when someone asks How do I...? or What is...? questions about game content.",
    "configuration": {
        "pattern": "game_knowledge"  # Index pattern - specifies which ES index to search
    },
}

response = requests.post(
    f"{KIBANA_URL}/api/agent_builder/tools", headers=headers, json=game_knowledge_tool
)
print(f"Game knowledge tool: {response.status_code}")

## Create the agent

The agent configuration defines:

1. Tool access (leaderboard, hero_stats, meta_report, game_knowledge)
2. Instructions on when to use each tool
3. Response style (friendly, concise)

The `instructions` field tells the LLM how to behave and when to use each tool.

In [ ]:
agent = {
    "id": "gaming_support_bot",
    "name": "Gaming Support Bot",
    "description": "A gaming community support bot that answers player questions about stats, heroes, and game mechanics.",
    "configuration": {
        "tools": [
            {"tool_ids": ["leaderboard", "hero_stats", "meta_report", "game_knowledge"]}
        ],
        "instructions": """You are a helpful gaming community bot. Answer player questions about:
- Player stats and leaderboards (use leaderboard tool)
- Hero performance and meta (use hero_stats and meta_report tools)
- Game mechanics and guides (use game_knowledge tool)

Be concise and friendly. Format leaderboards clearly with rankings.""",
    },
}

response = requests.post(
    f"{KIBANA_URL}/api/agent_builder/agents", headers=headers, json=agent
)
print(f"Agent created: {response.status_code}")

## Test the agent

Test the agent by sending a message through the `/converse` API endpoint.

By default, Agent Builder uses the [Elastic Managed LLM](https://www.elastic.co/docs/reference/kibana/connectors-kibana/elastic-managed-llm)—no connector configuration required. This makes your code portable across Elasticsearch deployments (Serverless, 9.2+) without connector ID changes.

> **Note:** To use a different LLM provider (OpenAI, Bedrock, Gemini), pass an optional `connector_id` parameter. Refer to [generative AI connectors](https://www.elastic.co/docs/reference/kibana/connectors-kibana/gen-ai-connectors) for setup.

In [ ]:
# Test with an analytics query (will use the meta_report ES|QL tool)
test_message = "Show me all heroes sorted by tier"

response = requests.post(
    f"{KIBANA_URL}/api/agent_builder/converse",
    headers=headers,
    json={"agent_id": "gaming_support_bot", "input": test_message},
    timeout=60,
)

print(f"Status: {response.status_code}")
if response.status_code == 200:
    result = response.json()
    print(
        f"\nAgent used tools: {[step.get('tool_id') for step in result.get('steps', []) if step.get('type') == 'tool_call']}"
    )
    print(f"\nResponse:\n{result.get('response', {}).get('message', 'No message')}")
else:
    print(f"Error: {response.text}")

## Additional tests

Test the other tool types to verify the agent selects the appropriate tool:

1. **Semantic search** - Game mechanics questions trigger the `game_knowledge` Index Search tool
2. **Dynamic parameter** - Hero-specific questions trigger `hero_stats` with the `?hero` parameter filled in

In [ ]:
# Test semantic search - should use the game_knowledge Index Search tool
# This demonstrates how the agent handles unstructured knowledge queries
test_message = "How do I get the dragon mount?"

response = requests.post(
    f"{KIBANA_URL}/api/agent_builder/converse",
    headers=headers,
    json={"agent_id": "gaming_support_bot", "input": test_message},
    timeout=60,
)

if response.status_code == 200:
    result = response.json()
    print(f"Question: {test_message}")
    print(
        f"Tool used: {[step.get('tool_id') for step in result.get('steps', []) if step.get('type') == 'tool_call']}"
    )
    print(f"\nResponse:\n{result.get('response', {}).get('message', 'No message')}")

In [ ]:
# Test dynamic parameter - should use hero_stats ES|QL tool with ?hero = "Mage"
# The agent extracts the hero name from natural language and fills the query parameter
test_message = "How good is Mage right now?"

response = requests.post(
    f"{KIBANA_URL}/api/agent_builder/converse",
    headers=headers,
    json={"agent_id": "gaming_support_bot", "input": test_message},
    timeout=60,
)

if response.status_code == 200:
    result = response.json()
    print(f"Question: {test_message}")
    print(
        f"Tool used: {[step.get('tool_id') for step in result.get('steps', []) if step.get('type') == 'tool_call']}"
    )
    print(f"\nResponse:\n{result.get('response', {}).get('message', 'No message')}")

## Next steps

Now you can connect this agent to Discord using the A2A client:

```bash
git clone https://github.com/llermaly/agentbuilder-a2a-discord
cd agentbuilder-a2a-discord
```

Create a `.env` file:
```
DISCORD_BOT_TOKEN=<your_bot_token>
AGENT_BUILDER_URL=https://<kibana_url>/api/agent_builder/a2a/gaming_support_bot
A2A_API_KEY=<your_api_key>
```

Run the bot:
```bash
uv venv && uv sync && uv run main.py
```

## Bidirectional: Giving the agent actions

Beyond answering questions, we can give Agent Builder the ability to trigger Discord actions. With a small modification to the Discord client, we can parse special tags in the agent's response and execute Discord commands.

For example, we added support for a `<poll>` tag. When the agent includes this in its response, the bot creates a native Discord poll.

In [ ]:
# Update agent with poll support
agent = {
    "id": "gaming_support_bot",
    "name": "Gaming Support Bot",
    "description": "A gaming community support bot that answers player questions about stats, heroes, and game mechanics.",
    "configuration": {
        "tools": [
            {"tool_ids": ["leaderboard", "hero_stats", "meta_report", "game_knowledge"]}
        ],
        "instructions": """You are a helpful gaming community bot. Answer player questions about:
- Player stats and leaderboards (use leaderboard tool)
- Hero performance and meta (use hero_stats and meta_report tools)
- Game mechanics and guides (use game_knowledge tool)

When discussing balance topics, create a poll for community input.
Use: <poll>Question|Option1|Option2|Option3</poll>

Be concise and friendly. Format leaderboards clearly with rankings.""",
    },
}

response = requests.put(
    f"{KIBANA_URL}/api/agent_builder/agents", headers=headers, json=agent
)
print(f"Agent updated: {response.status_code}")